### Implement Edge Detection

Write a Python function using OpenCV that takes an image file path as input, applies Canny edge detection on the image, and displays the original and edge-detected images side by side.

In [92]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

In [93]:
def canny(path):
    image = cv.imread(path)
    canny = cv.Canny(image,125,175)
    canny = cv.cvtColor(canny,cv.COLOR_GRAY2BGR)

    display_image = np.hstack((image,canny))

    plt.imshow(display_image[:,:,::-1])
    plt.axis(False)


### Face and Eye Detection

Create a function that detects faces and eyes in a given image using Haar cascades in OpenCV. The function should draw rectangles around detected faces and eyes and display the output image.

In [94]:
def face_d_eye(image):
    gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)

    face_cascade = cv.CascadeClassifier(cv.data.haarcascades + 'haarcascade_frontalface_default.xml')
    eye_cascade = cv.CascadeClassifier(cv.data.haarcascades + 'haarcascade_eye.xml')

    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.001, minNeighbors=5, minSize=(30, 30))

    for (x,y,w,h) in faces:
        face_img = cv.rectangle(image,(x,y),(x+w,y+h),(255,0,0),2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = face_img[y:y+h, x:x+w]
        eyes = eye_cascade.detectMultiScale(roi_gray)
        for (ex,ey,ew,eh) in eyes:
            cv.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)

    face_img = cv.rectangle(image,(x,y),(x+w,y+h),(255,0,0),2)
    plt.imshow(face_img[:,:,::-1])
    plt.axis(False)


###Image Cropping Based on Facial Features

Write a function that takes an image path as input and detects faces. If exactly one face is detected, return the cropped image of the face. Use Haar cascades for face detection.

In [95]:
def face_d_eye(image):
    gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)

    face_cascade = cv.CascadeClassifier(cv.data.haarcascades + 'haarcascade_frontalface_default.xml')
    

    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.001, minNeighbors=5, minSize=(30, 30))
    if len(faces)==1:
        x,y,w,h = faces[0]

    roi_color = image[y:y+h, x:x+w]
    plt.imshow(roi_color[:,:,::-1])
    plt.axis(False)


### Feature Matching with ORB
Create a Python script that uses ORB to detect and match features between two images. The script should display the matched keypoints on the output image.

In [96]:
def fetComp(path1,path2):
    image1 = cv.imread(path1, cv.IMREAD_GRAYSCALE)
    image2 = cv.imread(path2, cv.IMREAD_GRAYSCALE)

    orb = cv.ORB_create()

    keypoints1, descriptors1 = orb.detectAndCompute(image1, None)
    keypoints2, descriptors2 = orb.detectAndCompute(image2, None)
    
    bf = cv.BFMatcher(cv.NORM_HAMMING, crossCheck=True)

    matches = bf.match(descriptors1, descriptors2)

    matches = sorted(matches, key=lambda x: x.distance)

    image1_with_points = cv.cvtColor(image1, cv.COLOR_GRAY2BGR)
    image2_with_points = cv.cvtColor(image2, cv.COLOR_GRAY2BGR)



    for match in matches[:20]:
        
        pt1 = tuple(map(int, keypoints1[match.queryIdx].pt))
        pt2 = tuple(map(int, keypoints2[match.trainIdx].pt))

        a,b,c= np.random.rand(1,3)[0]*255
        a=int(a)
        b=int(b)
        c=int(c)
        
        cv.circle(image1_with_points, pt1, radius=7, color=(a,b,c), thickness=-1)
        cv.circle(image2_with_points, pt2, radius=7, color=(a,b,c), thickness=-1)


    combined_image = cv.hconcat([image1_with_points, image2_with_points])

    plt.figure(figsize=(12, 6))
    plt.imshow(cv.cvtColor(combined_image, cv.COLOR_BGR2RGB))
    plt.title('ORB Feature Detection: Marked Keypoints')
    plt.axis('off')
    plt.show()


### Applying Gaussian Blur for Noise Reduction
Write a function that applies a Gaussian blur to an image to reduce noise and displays both the original and blurred images.

In [97]:
def blur(image):
    blur = cv.blur(image,(11,11))
    display_image = np.hstack((image,blur))

    plt.imshow(display_image[:,:,::-1])
    plt.axis(False)


### Pyramid Transform for Image Scaling
Create a function that creates a pyramid of images (both up and down) for a given image and displays the results.

In [98]:

def display_image_pyramid(image_path, levels=3):

    image = cv.imread(image_path)
    if image is None:
        raise IOError(f"Error loading image from path: {image_path}")
    
    
    image_rgb = cv.cvtColor(image, cv.COLOR_BGR2RGB)

    pyramid_down = [image_rgb]
    pyramid_up = [image_rgb]

    for i in range(levels):
        image_rgb = cv.pyrDown(image_rgb)
        pyramid_down.append(image_rgb)

    image_rgb = pyramid_down[-1]
    for i in range(levels):
        image_rgb = cv.pyrUp(image_rgb)
        pyramid_up.append(image_rgb)

    plt.figure(figsize=(15, 8))
    total_images = len(pyramid_down) + len(pyramid_up) - 1
    for i, img in enumerate(pyramid_down + pyramid_up[1:], start=1):
        plt.subplot(2, total_images // 2 + 1, i)
        plt.imshow(img)
        plt.title(f"Level {i - 1}")
        plt.axis('off')

    plt.suptitle("Image Pyramid (Down and Up)", fontsize=16)
    plt.tight_layout()
    plt.show()



### Implement Harris Corner Detection in Python
Write a Python function using OpenCV that takes an image file as input and applies the Harris Corner Detection algorithm. Your function should display the original image with the detected corners marked. Include parameters to specify the block size, ksize, and free parameter for flexibility.

In [99]:
def edgeDet(image,blockSize,ksize,k):

    copy_img = image.copy()
    gray_image = cv.cvtColor(image, cv.COLOR_BGR2GRAY) 
    
    dst = cv.cornerHarris(gray_image, blockSize=blockSize, ksize=ksize, k=k) 
    
    dst = cv.dilate(dst, None) 
    copy_img[dst > 0.01 * dst.max()] = [0, 255, 0] 
    
    plt.imshow(copy_img[:,:,::-1])
    plt.axis(False)


### SIFT Keypoint Detection and Description
Create a function that reads an image, converts it to grayscale, and then applies the SIFT algorithm to detect keypoints and compute descriptors. Ensure the detected keypoints are visualized on the original image.

In [100]:
def detKeypoints(image):
    
    gray_image = cv.cvtColor(image, cv.COLOR_BGR2GRAY) 
    
    sift = cv.SIFT_create() 
    kp, des = sift.detectAndCompute(gray_image, None) 
    
    kp_image = cv.drawKeypoints(image, kp, None, color=( 
        0, 255, 0), flags=cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS) 
    
    plt.imshow(kp_image[:,:,::-1])
    plt.axis(False)


### Feature Matching using ORB
Develop a Python script that matches features between two images using the ORB algorithm. The script should display the matched features between the two images with lines connecting corresponding keypoints.

In [102]:
def matchFet(path1,path2):
    image1 = cv.imread(path1, cv.IMREAD_GRAYSCALE)
    image2 = cv.imread(path2, cv.IMREAD_GRAYSCALE)

    orb = cv.ORB_create()

    keypoints1, descriptors1 = orb.detectAndCompute(image1, None)
    keypoints2, descriptors2 = orb.detectAndCompute(image2, None)
    
    bf = cv.BFMatcher(cv.NORM_HAMMING, crossCheck=True)

    matches = bf.match(descriptors1, descriptors2)

    matches = sorted(matches, key=lambda x: x.distance)

    matched_image = cv.drawMatches(image1, keypoints1, image2, keypoints2, matches[:20], None, flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

    matched_image_rgb = cv.cvtColor(matched_image, cv.COLOR_BGR2RGB)

    plt.figure(figsize=(12, 6))
    plt.imshow(matched_image_rgb)
    plt.title('ORB Feature Matching')
    plt.axis('off')
    plt.show()


### Implement FAST Corner Detection
Write a Python function to implement the FAST corner detection algorithm. The function should accept an image and return the image with detected keypoints highlighted.

In [103]:
def cornerDet(image):
    gray_image = cv.cvtColor(image, cv.COLOR_BGR2GRAY) 
  
    fast = cv.FastFeatureDetector_create() 
    fast.setNonmaxSuppression(False) 

    kp = fast.detect(gray_image, None) 
    kp_image = cv.drawKeypoints(image, kp, None, color=(0, 255, 0)) 
    
    plt.imshow(kp_image[:,:,::-1]) 
    plt.axis(False)


Tough questions

### Custom Canny Edge Detector Implementation
Implement your own version of the Canny edge detection algorithm from scratch using Python (without using OpenCV functions). Your implementation should include:

Gaussian filtering for noise reduction.
Calculation of gradient magnitude and direction.
Non-maximum suppression.
Hysteresis thresholding. Your function should take an image as input and return an image with detected edges.

In [ ]:
def gaussian_kernel(size, sigma):

    k = size // 2
    x, y = np.meshgrid(np.arange(-k, k+1), np.arange(-k, k+1))
    kernel = (1 / (2 * np.pi * sigma**2)) * np.exp(-(x**2 + y**2) / (2 * sigma**2))
    return kernel / kernel.sum()

def sobel_filter(image, axis):

    if axis == 'x':
        kernel = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
    elif axis == 'y':
        kernel = np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]])
    else:
        raise ValueError("Invalid axis: choose 'x' or 'y'")
    return cv.filter2D(image, -1, kernel)

def custom_canny(image, low_threshold, high_threshold, sigma=1.0):

    kernel_size = int(6 * sigma + 1) | 1
    smoothed = cv.filter2D(image, -1, gaussian_kernel(kernel_size, sigma))

    Gx = sobel_filter(smoothed, 'x')
    Gy = sobel_filter(smoothed, 'y')
    magnitude = np.hypot(Gx, Gy) 
    direction = np.arctan2(Gy, Gx) 

    magnitude = (magnitude / magnitude.max()) * 255

    suppressed = np.zeros_like(magnitude)
    angle = np.rad2deg(direction) % 180 

    for x in range(1, image.shape[0] - 1):
        for y in range(1, image.shape[1] - 1):
            q, r = 255, 255
        
            if (0 <= angle[x, y] < 22.5) or (157.5 <= angle[x, y] <= 180):
                q = magnitude[x, y + 1]
                r = magnitude[x, y - 1]
            elif 22.5 <= angle[x, y] < 67.5:
                q = magnitude[x + 1, y - 1]
                r = magnitude[x - 1, y + 1]
            elif 67.5 <= angle[x, y] < 112.5:
                q = magnitude[x + 1, y]
                r = magnitude[x - 1, y]
            elif 112.5 <= angle[x, y] < 157.5:
                q = magnitude[x - 1, y - 1]
                r = magnitude[x + 1, y + 1]

            if magnitude[x, y] >= q and magnitude[x, y] >= r:
                suppressed[x, y] = magnitude[x, y]
            else:
                suppressed[x, y] = 0

    strong = (suppressed > high_threshold).astype(np.uint8)
    weak = ((suppressed >= low_threshold) & (suppressed <= high_threshold)).astype(np.uint8)

    edges = np.zeros_like(image)
    for x in range(1, image.shape[0] - 1):
        for y in range(1, image.shape[1] - 1):
            if weak[x, y]:
                if (strong[x - 1:x + 2, y - 1:y + 2].any()):
                    edges[x, y] = 255
            elif strong[x, y]:
                edges[x, y] = 255

    return edges

image = cv.imread('image.jpg', cv.IMREAD_GRAYSCALE)
edges = cv.cvtColor(custom_canny(image, low_threshold=50, high_threshold=100, sigma=1.0),cv.COLOR_GRAY2RGB)

plt.imshow(edges[:,::-1])
plt.axis(False)



### Multi-Scale Feature Detection
Create a function that applies multi-scale feature detection using the Laplacian of Gaussian (LoG) method. This function should take an image and a list of sigma values as input, and return an image or set of images showing detected features at the different scales.

In [ ]:
def laplacian_of_gaussian(image, sigmas):
    
    if len(image.shape) == 3:
        image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)

    image = image / 255.0

    log_images = []
    feature_images = []

    for sigma in sigmas:
        
        ksize = int(6 * sigma + 1)
        if ksize % 2 == 0:
            ksize += 1 

        blurred = cv.GaussianBlur(image, (ksize, ksize), sigma)

        log = cv.Laplacian(blurred, cv.CV_64F, ksize=ksize)

        zero_crossings = np.zeros_like(log, dtype=np.uint8)
        rows, cols = log.shape
        for i in range(1, rows - 1):
            for j in range(1, cols - 1):
                patch = log[i-1:i+2, j-1:j+2]
                if (patch.max() > 0 and patch.min() < 0):
                    zero_crossings[i, j] = 255

        log_images.append(log)
        feature_images.append(zero_crossings)

    return feature_images, log_images


if __name__ == "__main__":
    image = cv.imread("image.jpg", cv.IMREAD_GRAYSCALE)

    sigma_values = [1, 2, 3, 4]

    feature_images, log_images = laplacian_of_gaussian(image, sigma_values)

    for idx, (features, log_img) in enumerate(zip(feature_images, log_images)):
        cv.imshow(f"LoG Features (sigma={sigma_values[idx]})", features)
        cv.imshow(f"LoG Raw (sigma={sigma_values[idx]})", cv.convertScaleAbs(log_img))
    
    cv.waitKey(0)
    cv.destroyAllWindows()


## Bonus Question

Do your Research on how can we make a 3d model from the Refined frames.  

In [ ]:
### Write your answer here.